In [1]:
sc

In [2]:
csv = 'airports.csv'
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(csv)

In [3]:
df.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [4]:
df.show(15)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 01A|Afognak Lake, AK:...|
| 03A|Granite Mountain,...|
| 04A|Lik, AK: Lik Mini...|
| 05A|Little Squaw, AK:...|
| 06A|Kizhuyak, AK: Kiz...|
| 07A|Klawock, AK: Klaw...|
| 08A|Elizabeth Island,...|
| 09A|Homer, AK: August...|
| 1B1|Hudson, NY: Colum...|
| 1G4|Peach Springs, AZ...|
| 1N7|Blairstown, NJ: B...|
| 1NY|Penn Yan, NY: Pen...|
| 6B0|Middlebury, VT: M...|
| 7AK|Akun, AK: Akun Ai...|
| 8F3|Crosbyton, TX: Cr...|
+----+--------------------+
only showing top 15 rows



In [5]:
from pyspark.sql.types import *
customSchema = StructType([StructField('Code', StringType(), True),
                           StructField('Description', StringType(), True)])

df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load(csv, schema = customSchema).cache() 

In [6]:
df.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [7]:
df.select('Code').distinct().collect()

[Row(Code=u'A23'),
 Row(Code=u'BGM'),
 Row(Code=u'BOX'),
 Row(Code=u'BZT'),
 Row(Code=u'CCK'),
 Row(Code=u'CLQ'),
 Row(Code=u'CRS'),
 Row(Code=u'CSR'),
 Row(Code=u'FAV'),
 Row(Code=u'FIZ'),
 Row(Code=u'FMY'),
 Row(Code=u'GIS'),
 Row(Code=u'HYL'),
 Row(Code=u'ITJ'),
 Row(Code=u'JWY'),
 Row(Code=u'KEB'),
 Row(Code=u'KGL'),
 Row(Code=u'KLR'),
 Row(Code=u'KMU'),
 Row(Code=u'LEB'),
 Row(Code=u'LEN'),
 Row(Code=u'MDX'),
 Row(Code=u'MIZ'),
 Row(Code=u'NBC'),
 Row(Code=u'NWI'),
 Row(Code=u'OXC'),
 Row(Code=u'PEA'),
 Row(Code=u'PKE'),
 Row(Code=u'PMI'),
 Row(Code=u'QNE'),
 Row(Code=u'RKP'),
 Row(Code=u'SGT'),
 Row(Code=u'SKJ'),
 Row(Code=u'SUC'),
 Row(Code=u'TEU'),
 Row(Code=u'TNP'),
 Row(Code=u'UAB'),
 Row(Code=u'VWD'),
 Row(Code=u'WQN'),
 Row(Code=u'YUL'),
 Row(Code=u'AMF'),
 Row(Code=u'AVX'),
 Row(Code=u'BIP'),
 Row(Code=u'CA5'),
 Row(Code=u'DLG'),
 Row(Code=u'ELI'),
 Row(Code=u'GDT'),
 Row(Code=u'HAE'),
 Row(Code=u'HEL'),
 Row(Code=u'HTI'),
 Row(Code=u'IML'),
 Row(Code=u'INL'),
 Row(Code=u'

In [8]:
df.filter("Code = 'PUF'").select(df.Description).show(1)

+--------------------+
|         Description|
+--------------------+
|Pau, France: Pau ...|
+--------------------+



In [9]:
p = df.toPandas()
p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6323 entries, 0 to 6322
Data columns (total 2 columns):
Code           6323 non-null object
Description    6323 non-null object
dtypes: object(2)
memory usage: 98.9+ KB


### Otra forma mediante createDataFrame

In [10]:
import csv

from StringIO import StringIO
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, Row

def split(line):
    """
    Operator function for splitting a line on a delimiter.
    """
    reader = csv.reader(StringIO(line))
    return reader.next()

In [14]:
#leer fichero csv y establecemos las columnas que vamos a leer
rows = sc.textFile("airports.csv").map(split)
airportsRDD = rows.map(lambda c: Row(code=c[0], description=c[1]))

spark= SparkSession.builder.appName("Trabajando con Spark SQL").getOrCreate()

type(airportsRDD)

pyspark.rdd.PipelinedRDD

In [15]:
# registrar esquema
schema = spark.createDataFrame(airportsRDD)
schema.registerTempTable("airports")

select = spark.sql("SELECT * FROM airports WHERE code = 'GIS'")
print(select.count())
print(select.show())

1
+----+--------------------+
|code|         description|
+----+--------------------+
| GIS|Gisborne, New Zea...|
+----+--------------------+

None
